In [59]:
from requests import get
from bs4 import BeautifulSoup
import os
import pandas as pd
import pprint
import env
from time import sleep
from random import randint

pp = pprint.PrettyPrinter(indent=4)

import re
from selenium.webdriver.chrome.service import Service

from selenium.webdriver.support.ui import WebDriverWait

from selenium.webdriver.support import expected_conditions as EC


import codecs


import hmac
import json 





In [60]:




def get_article_content(some_article):
    '''
    grab the content out of the beautiful soup object
    for each article in our overall set of articles
    '''
    output = {}
    output['headline'] = some_article.find('h2').text.strip()
    output['date'], output['author'], output['content'] = \
    [thing.text.strip() for thing in some_article.find_all('p')]
    return output


# print(soup.article.text)
def get_codeup_article_content(soup):
    splitup=soup.article.text.strip().splitlines()
    title=splitup[0]
    splitup[1]=splitup[1].split('|')
    category=splitup[1][1].strip()
    content=''.join(splitup[2:])
    dicta={'title': title,'content':content }  
   
    return dicta



def codeuparticles_df(urllist):
    codeuparticles=[]
    headers = {'User-Agent': 'Codeup Data Science'}
    for i in range(0,len(urllist)):
        soup=BeautifulSoup(get(urllist[i],headers=headers).content,'html.parser')
        dicta=get_codeup_article_content(soup)
        codeuparticles.append(dicta)

    codeuparticles=pd.DataFrame(codeuparticles)
    return codeuparticles

blogurls=[]
blogurls.append('https://codeup.com/codeup-news/dei-report/')
blogurls.append('https://codeup.com/codeup-news/diversity-and-inclusion-award/')
blogurls.append('https://codeup.com/featured/financing-career-transition/')
blogurls.append('https://codeup.com/tips-for-prospective-students/tips-for-women/')
blogurls.append('https://codeup.com/cloud-administration/cloud-computing-and-aws/')




headers = {'User-Agent': 'Codeup Data Science'} # Some websites don't accept the pyhon-requests default user-agent
soup=BeautifulSoup(get(blogurls[0],headers=headers).content,'html.parser')
soup.title.text
# print(soup.article.text)

   
codeuparticles=codeuparticles_df(blogurls)
codeuparticles






url='https://blog.inshorts.com/'



def souphtmltags(soup,toprint=False):
    tagset=set()
    attributes_set=set()
    aggtagdict={}

    for tag in soup.findAll(True):
        x=list(tag.attrs.keys())
        c=len(attributes_set)
        attributes_set.update(x)
        d=len(attributes_set)
        a=len(tagset)
        tagset.update([tag.name])
        b=len(tagset)
        if (a!=b) or (c!=d):
            if tag.name in list(aggtagdict.keys()):
                aggtagdict[tag.name].update(x)
            else:
                aggtagdict.update({tag.name:(set(x))})

    x=(list(aggtagdict.values()))
    a=set()
    x=list(filter((lambda i:len(i)>0),x))
    for i in x:
        a|=i

    attsdiff=a.symmetric_difference(attributes_set)
    keysetdiff=set(aggtagdict.keys()).symmetric_difference(tagset)


    
            


      
            
       
       
   

    if len(attsdiff.symmetric_difference(keysetdiff))!=0:
        print('logic error')
        
    pp.pprint(aggtagdict)
    return aggtagdict



aggtagdict=souphtmltags(soup)







newsurls=[]
for i in (soup.find_all('article')):
    newsurls.append(i.p.a['href'])
newsurls



aggtagdict=souphtmltags(soup)
soups=BeautifulSoup(get(newsurls[0],headers=headers).content,'html.parser')
aggtagdict=souphtmltags(soups)






selection=soups.find_all('article')
hrefs=[]
titles=[]
for i in selection:    
    href=i.h2.a['href']
    title=i.h2.string
    hrefs.append(href)
    titles.append(title)

articledict=dict(zip(titles,hrefs))
toget=list(articledict.values())[0]
soups=BeautifulSoup(get(toget,headers=headers).content,'html.parser')


selection=soups.find_all({'article':'p'})
for i in selection:
    print(i.text)
    print(i.div)

# soups.find_all(_class=)

jlist=[]
for j in selection:
        jlist.extend([(j.text)])

jlist[0]=jlist[0].split()
{'category': jlist[0][1]}
jlist
           
 










from copy import copy


a=set([1,2,3,4,5]) 
b=set([2,6,8,23,1,77,56]) 
c=set([34,6,55,1,2,3])
d=set([90,6,5,4,3,1,8,0]) 
e=set([88,90,1,4,8,1,5,4,3,6,0]) 

setlist=[
a,
b,
c,
d,
e]



setunion=set()
setunion.update(*setlist)
dictofsets={}
for k,i in enumerate(setlist):
    '''    
    For set i in list of sets where the aggregate is our local universe  
    '''
    newset=set()
    isymdiff=i.symmetric_difference(setunion)

    # percent=1-(len(isymdiff)/len(setunion))
    # print((percent))
    templist=copy(setlist)
    templist.pop(templist.index(i))
    newset.update(*templist)   
    uniuniquie_to_i=i.difference(newset)
    unique_ilen=(len(uniuniquie_to_i))
    i=list(i)
    uniuniquie_to_i=list(uniuniquie_to_i)
    unique_ilen=[unique_ilen]
    isymdiff=list(isymdiff)
    i.sort()
    uniuniquie_to_i.sort()
    unique_ilen.sort()
    isymdiff.sort()     
    dictofsets.update({f'set_{k}':i,f'unique_to_{k}_wrt_aggregate':uniuniquie_to_i,f'count_unique_to_{k}_wrt_aggregate':unique_ilen,f'unique_to_pop_wrt_{k}':isymdiff})

   
   
pp.pprint(dictofsets)

# setintersection=setunion.intersection_update(*setlist)
# setintersection




# setintersection

## This set of elements that are common within our universe 


# setunion-aggintersection
## These are unique overall aka our aggregate set difference





##venn diagrams with wordcloud as the fill.











{   'a': {   'aria-label',
             'class',
             'data-cli-action',
             'data-cli_action',
             'data-readless-text',
             'data-readmore-text',
             'data-target',
             'data-toggle',
             'href',
             'id',
             'rel',
             'role',
             'style',
             'tabindex',
             'target'},
    'article': {'class', 'id'},
    'body': {'class'},
    'br': set(),
    'button': {'class', 'id', 'type'},
    'defs': set(),
    'div': {   'aria-hidden',
               'aria-labelledby',
               'class',
               'data-et-multi-view',
               'data-et-multi-view-load-phone-hidden',
               'data-et-multi-view-load-tablet-hidden',
               'data-nosnippet',
               'id',
               'role',
               'tabindex'},
    'fecolormatrix': {'color-interpolation-filters', 'values', 'type'},
    'fecomponenttransfer': {'color-interpolation-filters'},
    'f

In [61]:





def souphtmltags(soup,toprint=False):
    tagset=set()
    attributes_set=set()
    aggtagdict={}

    for tag in soup.findAll(True):
        x=list(tag.attrs.keys())
        c=len(attributes_set)
        attributes_set.update(x)
        d=len(attributes_set)
        a=len(tagset)
        tagset.update([tag.name])
        b=len(tagset)
        if (a!=b) or (c!=d):
            if tag.name in list(aggtagdict.keys()):
                aggtagdict[tag.name].update(x)
            else:
                aggtagdict.update({tag.name:(set(x))})

    x=(list(aggtagdict.values()))
    a=set()
    x=list(filter((lambda i:len(i)>0),x))
    for i in x:
        a|=i

    attsdiff=a.symmetric_difference(attributes_set)
    keysetdiff=set(aggtagdict.keys()).symmetric_difference(tagset)


    
            


      
            
       
       
   

    if len(attsdiff.symmetric_difference(keysetdiff))!=0:
        print('logic error')
        
    pp.pprint(aggtagdict)
    return aggtagdict


In [4]:

# url1='https://github.com/trending/python?since=daily&spoken_language_code=en'
# url2='https://github.com/trending/c?since=daily&spoken_language_code=en'
# url3='https://github.com/trending/java?since=daily&spoken_language_code=en'

# urllist=[url1,url2,url3]
# soup=BeautifulSoup(get(url=url2,headers=headers).content,'html.parser')

# # aggtagdict=souphtmltags(soup,toprint=False)
# # aggtagdict


In [5]:
# # Declare the filter function
# def Filter(datalist):
#     # Search data based on regular expression in the list
#     return [val for val in datalist
#         if re.search(r'^((?!network).)*$', val)]

# # selection=soups.find_all('article')
# # hrefs=[]
# # titles=[]
# # for i in selection:    
# #     href=i.h2.a['href']
# #     title=i.h2.string
# #     hrefs.append(href)
# #     titles.append(title)

# # articledict=dict(zip(titles,hrefs))
# # toget=list(articledict.values())[0]
# # soups=BeautifulSoup(get(toget,headers=headers).content,'html.parser')
# urllistactual=[]
# for url in urllist:
#    soup=BeautifulSoup(get(url=url2,headers=headers).content,'html.parser')   
#    selection=soup.find_all('a',href=True,class_="Link--muted d-inline-block mr-3")
#    for i in selection:
    
#       urllistactual.append(i['href'])
 

In [ ]:
##Project stuff

In [68]:
#This ends up taking the top 150 results from github from three independent languages where the readme is written in english and the license is MIT 
# the last part was just a heuristic to help ensure that the results are as professional as possible and hopefully have a read.me
# the sleep was make the query look more like a human clicking through it ( although it was not that sophisticated)
# the code is also implemented so it loops through each of the urls before switching the page number, again this was to create the illusion of seperate searches 


urls=[]
url1=[]
url2=[]
url3=[]
# Here we generate all our urls then we zip them into a tuple
for i in range(1,51):  
    url1.append(f"https://github.com/search?l=&p={str(i)}&q=language%3AC+license%3Amit&ref=advsearch&type=Repositories&spoken_language_code=en")
    url2.append(f'https://github.com/search?l=&p={str(i)}&q=language%3APython+license%3Amit&ref=advsearch&type=Repositories&spoken_language_code=en')
    url3.append(f"https://github.com/search?l=&p={str(i)}&q=language%3AJava+license%3Amit&ref=advsearch&type=Repositories&spoken_language_code=en")

tempurls=list(zip(url1,url2,url3))
tempurls


[('https://github.com/search?l=&p=1&q=language%3AC+license%3Amit&ref=advsearch&type=Repositories&spoken_language_code=en',
  'https://github.com/search?l=&p=1&q=language%3APython+license%3Amit&ref=advsearch&type=Repositories&spoken_language_code=en',
  'https://github.com/search?l=&p=1&q=language%3AJava+license%3Amit&ref=advsearch&type=Repositories&spoken_language_code=en'),
 ('https://github.com/search?l=&p=2&q=language%3AC+license%3Amit&ref=advsearch&type=Repositories&spoken_language_code=en',
  'https://github.com/search?l=&p=2&q=language%3APython+license%3Amit&ref=advsearch&type=Repositories&spoken_language_code=en',
  'https://github.com/search?l=&p=2&q=language%3AJava+license%3Amit&ref=advsearch&type=Repositories&spoken_language_code=en'),
 ('https://github.com/search?l=&p=3&q=language%3AC+license%3Amit&ref=advsearch&type=Repositories&spoken_language_code=en',
  'https://github.com/search?l=&p=3&q=language%3APython+license%3Amit&ref=advsearch&type=Repositories&spoken_language_cod

In [70]:
for u in tempurls:
        sleep(randint(3,5))
        for ui in u:

            soup=BeautifulSoup(get(url=ui,headers=headers).content,'html.parser')
            # print(i) 
            # print(u)
            # # Uncomment above if you need to see the urls as they are being populated
            data=soup.find_all('a',class_='v-align-middle') 
            for d in data:
                urls.append(d.string)
              

In [71]:

# newurls=[]
# for i in urllistactual:
#     i=i.replace('//','')
#     i=re.sub(r'((network))|((members))|((stargazers))','', i,  count=0, flags=0)
#     i=i=re.sub(r'((//))','/', i,  count=0, flags=0)
#     newurls.append(i)
  

len(urls)

270

In [72]:
"""
A module for obtaining repo readme and language data from the github API.
Before using this module, read through it, and follow the instructions marked
TODO.
After doing so, run it like this:
    python acquire.py
To create the `data.json` file that contains the data.
"""
import os
import json
from typing import Dict, List, Optional, Union, cast
import requests

from env import github_token, github_username

# TODO: Make a github personal access token.
#     1. Go here and generate a personal access token: https://github.com/settings/tokens
#        You do _not_ need select any scopes, i.e. leave all the checkboxes unchecked
#     2. Save it in your env.py file under the variable `github_token`
# TODO: Add your github username to your env.py file under the variable `github_username`
# TODO: Add more repositories to the `REPOS` list below.

REPOS = urls

headers = {"Authorization": f"token {github_token}", "User-Agent": github_username}

if headers["Authorization"] == "token " or headers["User-Agent"] == "":
    raise Exception(
        "You need to follow the instructions marked TODO in this script before trying to use it"
    )


def github_api_request(url: str) -> Union[List, Dict]:
    response = requests.get(url, headers=headers)
    response_data = response.json()
    if response.status_code != 200:
        raise Exception(
            f"Error response from github api! status code: {response.status_code}, "
            f"response: {json.dumps(response_data)}"
        )
    return response_data


def get_repo_language(repo: str) -> str:
    url = f"https://api.github.com/repos/{repo}"
    repo_info = github_api_request(url)
    if type(repo_info) is dict:
        repo_info = cast(Dict, repo_info)
        if "language" not in repo_info:
            raise Exception(
                "'language' key not round in response\n{}".format(json.dumps(repo_info))
            )
        return repo_info["language"]
    raise Exception(
        f"Expecting a dictionary response from {url}, instead got {json.dumps(repo_info)}"
    )


def get_repo_contents(repo: str) -> List[Dict[str, str]]:
    url = f"https://api.github.com/repos/{repo}/contents/"
    contents = github_api_request(url)
    if type(contents) is list:
        contents = cast(List, contents)
        return contents
    raise Exception(
        f"Expecting a list response from {url}, instead got {json.dumps(contents)}"
    )


def get_readme_download_url(files: List[Dict[str, str]]) -> str:
    """
    Takes in a response from the github api that lists the files in a repo and
    returns the url that can be used to download the repo's README file.
    """
    for file in files:
        if file["name"].lower().startswith("readme"):
            return file["download_url"]
    return ""


def process_repo(repo: str) -> Dict[str, str]:
    """
    Takes a repo name like "gocodeup/codeup-setup-script" and returns a
    dictionary with the language of the repo and the readme contents.
    """
    contents = get_repo_contents(repo)
    readme_download_url = get_readme_download_url(contents)
    if readme_download_url == "":
        readme_contents = ""
    else:
        readme_contents = requests.get(readme_download_url).text
    return {
        "repo": repo,
        "language": get_repo_language(repo),
        "readme_contents": readme_contents,
    }


def scrape_github_data() -> List[Dict[str, str]]:
    """
    Loop through all of the repos and process them. Returns the processed data.
    """
    return [process_repo(repo) for repo in REPOS]


if __name__ == "__main__":
    data = scrape_github_data()
    json.dump(data, open("data.json", "w"), indent=1)



